In [1]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess

In [2]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

True

In [2]:
from keras.layers import Activation,Convolution2D, Dropout,Conv2D
from keras.layers import AveragePooling2D,BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np
import scipy.misc
#from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

Using TensorFlow backend.


In [3]:
def convenet():
    model = Sequential()
    model.add(layers.Conv2D(32,(3,3),activation='relu',
                       input_shape=(150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [4]:
def CNN(input_shape,num_classes=1):
    model = Sequential()
    model.add(Convolution2D(filters=16, kernel_size=(7, 7), padding='same',
                            name='image_array', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=16, kernel_size=(7, 7), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=32, kernel_size=(5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(.5))

    model.add(Convolution2D(filters=256, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(filters=num_classes, kernel_size=(3, 3), padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('sigmoid',name='predictions'))
    return model

In [5]:
model = CNN((150,150,3))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_array (Conv2D)         (None, 150, 150, 16)      2368      
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 16)      64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 16)      12560     
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 150, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 150, 150, 16)      0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 75, 75, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 75, 16)        0         
__________

In [7]:
INIT_LR = 1e-4

In [8]:
model.compile(optimizer=optimizers.Adam(INIT_LR), loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_generator = train_datagen.flow_from_directory(
        'cats_and_dogs_small/train',
        target_size = (150,150),
        batch_size = 32,
        class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(
        'cats_and_dogs_small/validation',
        target_size = (150,150),
        batch_size = 32,
        class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 1500 images belonging to 2 classes.


In [12]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=50) #50

Epoch 1/50
100/100 [==============================] - 605s 6s/step - loss: 0.6711 - acc: 0.5781 - val_loss: 0.6684 - val_acc: 0.5871
Epoch 2/50
100/100 [==============================] - 581s 6s/step - loss: 0.6578 - acc: 0.6125 - val_loss: 0.6800 - val_acc: 0.5401
Epoch 3/50
100/100 [==============================] - 578s 6s/step - loss: 0.6529 - acc: 0.6156 - val_loss: 0.6365 - val_acc: 0.6197
Epoch 4/50
100/100 [==============================] - 579s 6s/step - loss: 0.6384 - acc: 0.6284 - val_loss: 0.7449 - val_acc: 0.4781
Epoch 5/50
100/100 [==============================] - 577s 6s/step - loss: 0.6232 - acc: 0.6525 - val_loss: 0.9109 - val_acc: 0.4793
Epoch 6/50
100/100 [==============================] - 577s 6s/step - loss: 0.6121 - acc: 0.6697 - val_loss: 0.5654 - val_acc: 0.6880
Epoch 7/50
100/100 [==============================] - 579s 6s/step - loss: 0.6087 - acc: 0.6600 - val_loss: 0.7353 - val_acc: 0.5877
Epoch 8/50
100/100 [==============================] - 576s 6s/step - 

In [27]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.figure(figsize= (6,6))
plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss,'b',label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

<Figure size 600x600 with 1 Axes>

In [ ]:
plt.clf()

acc = history.history['acc']
val_acc = history.history['val_acc']

plt.figure(figsize= (6,6))
plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc,'b',label='Validation acc')
plt.title('Training and validaion accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
'''test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(200, 200),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)'''